In [2]:
#Import libraries
from datetime import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from scipy.stats import norm
from scipy import stats
import scipy as sp
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import f_regression
sns.set()
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings("ignore")
plt.style.use('seaborn')
%matplotlib inline
pd.set_option('display.max_columns', 300)

In [3]:
#read in data
df = pd.read_csv('listingsclean.csv', index_col=0)

## Data Cleaning
This note book will continue cleaning outliers and removing features that are not usable
for our project

In [4]:
df.head()

,experiences_offered,host_response_rate,host_is_superhost,host_total_listings_count,neighbourhood_cleansed,city,zipcode,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,instant_bookable,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
958,none,100%,t,1.0,Western Addition,San Francisco,94117,37.76931,-122.43386,Apartment,Entire home/apt,3,1.0,1.0,2.0,NaN,$170.00,"$1,120.00","$4,200.00",$100.00,$100.00,2,$25.00,1,30,217,97.0,f,1.74
3850,none,100%,t,2.0,Inner Sunset,San Francisco,94131,37.75402,-122.45805,House,Private room,2,1.0,1.0,1.0,NaN,$99.00,NaN,NaN,$0.00,$10.00,2,$20.00,1,5,160,94.0,f,1.28
5858,none,80%,f,2.0,Bernal Heights,San Francisco,94110,37.74511,-122.42102,Apartment,Entire home/apt,5,1.0,2.0,3.0,NaN,$235.00,"$1,600.00","$5,500.00",NaN,$100.00,2,$0.00,30,60,111,98.0,f,0.87
7918,none,86%,t,10.0,Haight Ashbury,San Francisco,94117,37.76669,-122.45250,Apartment,Private room,2,4.0,1.0,1.0,NaN,$65.00,$485.00,"$1,685.00",$200.00,$50.00,1,$12.00,32,60,18,86.0,f,0.15
8142,none,86%,t,10.0,Haight Ashbury,San Francisco,94117,37.76487,-122.45183,Apartment,Private room,2,4.0,1.0,1.0,NaN,$65.00,$490.00,"$1,685.00",$200.00,$50.00,1,$12.00,32,90,8,93.0,f,0.13


In [5]:
#checking column types
df.dtypes

experiences_offered           object
host_response_rate            object
host_is_superhost             object
host_total_listings_count    float64
neighbourhood_cleansed        object
city                          object
zipcode                       object
latitude                     float64
longitude                    float64
property_type                 object
room_type                     object
accommodates                   int64
bathrooms                    float64
bedrooms                     float64
beds                         float64
square_feet                  float64
price                         object
weekly_price                  object
monthly_price                 object
security_deposit              object
cleaning_fee                  object
guests_included                int64
extra_people                  object
minimum_nights                 int64
maximum_nights                 int64
number_of_reviews              int64
review_scores_rating         float64
i

In [6]:
#checking dataframe shape
df.shape

(8111, 29)

In [7]:
df.isnull().sum()

experiences_offered             0
host_response_rate            927
host_is_superhost               8
host_total_listings_count       8
neighbourhood_cleansed          0
city                           10
zipcode                       245
latitude                        0
longitude                       0
property_type                   0
room_type                       0
accommodates                    0
bathrooms                      12
bedrooms                        4
beds                            9
square_feet                  7987
price                           0
weekly_price                 7042
monthly_price                7067
security_deposit             1692
cleaning_fee                  924
guests_included                 0
extra_people                    0
minimum_nights                  0
maximum_nights                  0
number_of_reviews               0
review_scores_rating         1651
instant_bookable                0
reviews_per_month            1605
dtype: int64

In [6]:
#looking at the City type column
df['city'].value_counts()

San Francisco                  8059
Daly City                        34
San Francisco                     3
Brisbane                          1
San Francisco, Hayes Valley       1
San Jose                          1
旧金山                               1
Noe Valley - San Francisco        1
Name: city, dtype: int64

In [7]:
#We are focusing just on SF so dropping where city is not SF
df= df[df['city']=='San Francisco']
df.drop(['city'],axis=1,inplace = True)

In [8]:
#looking at the square feet column
df['square_feet'].isna().sum()

7935

In [9]:
#We are missing 95% of the square feet data so dropping this column
df.drop(['square_feet'],axis=1,inplace = True)

In [10]:
#Checking daily price
df['price'].isna().sum()

0

In [11]:
#Checking Weekly Price
df['weekly_price'].isna().sum()

6992

In [12]:
#Checking Weekly Price

df['monthly_price'].isna().sum()

7017

In [ ]:
#dropping weekly and monthly price
df.drop(['monthly_price','weekly_price'],axis=1,inplace = True)


We are dropping some columns with large amounts of null values and we don't think they
make intuitive sense

In [14]:
df.drop(['zipcode','security_deposit','cleaning_fee','reviews_per_month'],axis=1,inplace = True)

Most listings do not have a weekly or a monthly price so we'll be using Daily Price as our outcome varialbe

In [16]:
df.dropna(subset=['bathrooms','bedrooms','host_total_listings_count','beds'],inplace=True) 

In [17]:
df.isnull().sum()

experiences_offered             0
host_response_rate            913
host_is_superhost               0
host_total_listings_count       0
neighbourhood_cleansed          0
latitude                        0
longitude                       0
property_type                   0
room_type                       0
accommodates                    0
bathrooms                       0
bedrooms                        0
beds                            0
price                           0
guests_included                 0
extra_people                    0
minimum_nights                  0
maximum_nights                  0
number_of_reviews               0
review_scores_rating         1631
instant_bookable                0
dtype: int64

In [18]:
df['experiences_offered'].value_counts()

none    8027
Name: experiences_offered, dtype: int64

In [19]:
df.drop(['experiences_offered'],axis=1,inplace = True)


In [20]:
df['room_type'].value_counts()

Entire home/apt    4730
Private room       2858
Shared room         232
Hotel room          207
Name: room_type, dtype: int64

## Cleaning t/f into 1/0

In [21]:
df['host_is_superhost'] = np.where(df['host_is_superhost']=='t', 1, 0)

In [22]:
df['host_is_superhost'].value_counts()

0    4507
1    3520
Name: host_is_superhost, dtype: int64

In [23]:
df['instant_bookable'] = np.where(df['instant_bookable']=='t', 1, 0)

In [24]:
df['instant_bookable'].value_counts()

0    4717
1    3310
Name: instant_bookable, dtype: int64

In [25]:
df.head()

,host_response_rate,host_is_superhost,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,instant_bookable
id,,,,,,,,,,,,,,,,,,,,
958,100%,1,1.0,Western Addition,37.76931,-122.43386,Apartment,Entire home/apt,3,1.0,1.0,2.0,$170.00,2,$25.00,1,30,217,97.0,0
3850,100%,1,2.0,Inner Sunset,37.75402,-122.45805,House,Private room,2,1.0,1.0,1.0,$99.00,2,$20.00,1,5,160,94.0,0
5858,80%,0,2.0,Bernal Heights,37.74511,-122.42102,Apartment,Entire home/apt,5,1.0,2.0,3.0,$235.00,2,$0.00,30,60,111,98.0,0
7918,86%,1,10.0,Haight Ashbury,37.76669,-122.45250,Apartment,Private room,2,4.0,1.0,1.0,$65.00,1,$12.00,32,60,18,86.0,0
8142,86%,1,10.0,Haight Ashbury,37.76487,-122.45183,Apartment,Private room,2,4.0,1.0,1.0,$65.00,1,$12.00,32,90,8,93.0,0


Replacing ratings NA values by average rating in the same neighborhood

In [26]:
df.groupby('neighbourhood_cleansed')['review_scores_rating'].mean()

neighbourhood_cleansed
Bayview                  94.083832
Bernal Heights           96.817416
Castro/Upper Market      97.818919
Chinatown                94.654762
Crocker Amazon           92.513514
Diamond Heights          98.187500
Downtown/Civic Center    90.798403
Excelsior                95.169697
Financial District       94.774194
Glen Park                97.440678
Golden Gate Park         95.750000
Haight Ashbury           95.563636
Inner Richmond           94.815642
Inner Sunset             96.540541
Lakeshore                93.777778
Marina                   95.696552
Mission                  96.132800
Nob Hill                 93.288557
Noe Valley               97.017007
North Beach              95.094488
Ocean View               95.521739
Outer Mission            96.128205
Outer Richmond           95.660377
Outer Sunset             95.289377
Pacific Heights          96.515625
Parkside                 96.440678
Potrero Hill             97.248780
Presidio                 97.0000

In [27]:
df["review_scores_rating"] = df.groupby('neighbourhood_cleansed')['review_scores_rating'].transform(lambda x: x.fillna(x.mean()))


In [28]:
df.isnull().sum()

host_response_rate           913
host_is_superhost              0
host_total_listings_count      0
neighbourhood_cleansed         0
latitude                       0
longitude                      0
property_type                  0
room_type                      0
accommodates                   0
bathrooms                      0
bedrooms                       0
beds                           0
price                          0
guests_included                0
extra_people                   0
minimum_nights                 0
maximum_nights                 0
number_of_reviews              0
review_scores_rating           1
instant_bookable               0
dtype: int64

In [29]:
df.groupby('neighbourhood_cleansed')['review_scores_rating'].mean()

neighbourhood_cleansed
Bayview                  94.083832
Bernal Heights           96.817416
Castro/Upper Market      97.818919
Chinatown                94.654762
Crocker Amazon           92.513514
Diamond Heights          98.187500
Downtown/Civic Center    90.798403
Excelsior                95.169697
Financial District       94.774194
Glen Park                97.440678
Golden Gate Park         95.750000
Haight Ashbury           95.563636
Inner Richmond           94.815642
Inner Sunset             96.540541
Lakeshore                93.777778
Marina                   95.696552
Mission                  96.132800
Nob Hill                 93.288557
Noe Valley               97.017007
North Beach              95.094488
Ocean View               95.521739
Outer Mission            96.128205
Outer Richmond           95.660377
Outer Sunset             95.289377
Pacific Heights          96.515625
Parkside                 96.440678
Potrero Hill             97.248780
Presidio                 97.0000

In [30]:
df.dropna(subset=['review_scores_rating'],inplace=True) 
df.isnull().sum()

host_response_rate           912
host_is_superhost              0
host_total_listings_count      0
neighbourhood_cleansed         0
latitude                       0
longitude                      0
property_type                  0
room_type                      0
accommodates                   0
bathrooms                      0
bedrooms                       0
beds                           0
price                          0
guests_included                0
extra_people                   0
minimum_nights                 0
maximum_nights                 0
number_of_reviews              0
review_scores_rating           0
instant_bookable               0
dtype: int64

In [8]:
df.head()

,experiences_offered,host_response_rate,host_is_superhost,host_total_listings_count,neighbourhood_cleansed,city,zipcode,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,instant_bookable,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
958,none,100%,t,1.0,Western Addition,San Francisco,94117,37.76931,-122.43386,Apartment,Entire home/apt,3,1.0,1.0,2.0,NaN,$170.00,"$1,120.00","$4,200.00",$100.00,$100.00,2,$25.00,1,30,217,97.0,f,1.74
3850,none,100%,t,2.0,Inner Sunset,San Francisco,94131,37.75402,-122.45805,House,Private room,2,1.0,1.0,1.0,NaN,$99.00,NaN,NaN,$0.00,$10.00,2,$20.00,1,5,160,94.0,f,1.28
5858,none,80%,f,2.0,Bernal Heights,San Francisco,94110,37.74511,-122.42102,Apartment,Entire home/apt,5,1.0,2.0,3.0,NaN,$235.00,"$1,600.00","$5,500.00",NaN,$100.00,2,$0.00,30,60,111,98.0,f,0.87
7918,none,86%,t,10.0,Haight Ashbury,San Francisco,94117,37.76669,-122.45250,Apartment,Private room,2,4.0,1.0,1.0,NaN,$65.00,$485.00,"$1,685.00",$200.00,$50.00,1,$12.00,32,60,18,86.0,f,0.15
8142,none,86%,t,10.0,Haight Ashbury,San Francisco,94117,37.76487,-122.45183,Apartment,Private room,2,4.0,1.0,1.0,NaN,$65.00,$490.00,"$1,685.00",$200.00,$50.00,1,$12.00,32,90,8,93.0,f,0.13


## Exporting Dropped Columns

In [11]:
df.to_csv('listingscleanFINAL.csv')